In [1]:
cd ..

/Users/dorislee/Desktop/Research/covid/covid19-vis


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/quarantine-activity-US-Apr16-long.csv")

In [4]:
import numpy as np

In [5]:
from chartlib.utils import create_lockdown_type, split_into_list, str2emo

In [6]:
# START modified from _ingest_usa_quarantine_df
quarantine_csv = "data/quarantine-activity-US-Apr16-long.csv"
quarantine_df = pd.read_csv(quarantine_csv)
groupcol = 'Province_State'

quarantine_df = quarantine_df.rename(columns={'State': 'Province_State', 'Effective Date': 'lockdown_date','Coverage.type':'Coverage'})
quarantine_df = quarantine_df.sort_values('Coverage', ascending=True)
quarantine_df['lockdown_type'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 0), axis=1)
quarantine_df['emoji_string'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 1), axis=1)
quarantine_df['lockdown_type'].replace('', np.nan, inplace=True)

quarantine_df = quarantine_df.dropna(subset=['lockdown_type'])
# #TODO: this is where the columns get dropped
# quarantine_df = quarantine_df.groupby(['lockdown_date', 'Province_State']).agg({
#     'lockdown_type': lambda col: '; '.join(col),
#     'emoji_string': lambda col: ''.join(col),
#     'population_size': lambda col:col,
#     'Coverage.location': lambda col:col
# }).reset_index()

quarantine_df.loc[quarantine_df.lockdown_type=="Regional Border Closure/Visitor Quarantine","emoji_string"]="t"#bugfix
# Breaking up emoji into separate rows for vertical stacking
quarantine_df.emoji_string = quarantine_df.emoji_string.apply(split_into_list)

quarantine_df = quarantine_df.explode(column='emoji_string')
quarantine_df['Coverage'] = quarantine_df.emoji_string.apply(
    lambda x: 'Statewide' if str(x).isupper() else 'Regional'
)
quarantine_df = quarantine_df.sort_values('Coverage', ascending=False)
quarantine_df.emoji_string = quarantine_df.emoji_string.str.lower()
quarantine_df['emoji'] = quarantine_df['emoji_string'].map(str2emo)
quarantine_df['event_index'] = quarantine_df.groupby(['Province_State', 'lockdown_date']).cumcount()

# # quarantine_cols = [
# #     groupcol, 'lockdown_date', 'lockdown_type', 'emoji', 'emoji_string', 'event_index', 'Coverage','Coverage.location'
# # ]
quarantine_cols = ['Province_State', 'Coverage', 'Coverage.location', 'lockdown_date','population_size','lockdown_type', 'emoji_string', 'emoji',
       'event_index']
quarantine_df = quarantine_df[quarantine_cols]

# # END modified from _ingest_usa_quarantine_df

In [7]:
quarantine_df_orig = pd.read_csv(quarantine_csv)

state_populations = quarantine_df_orig[quarantine_df_orig["Coverage.type"]=="State-wide"][["State","population_size"]].drop_duplicates()
# state_populations = state_populations.set_index(state_populations.State)
# state_populations = state_populations.drop(columns="State")
# state_populations_lookup= state_populations.population_size.to_dict()

In [8]:
state_populations = state_populations.rename(columns={'State': 'Province_State',"population_size":"state_population_size"})

In [9]:
#https://www.census.gov/quickfacts/PR
state_populations.loc[state_populations["Province_State"]=="Puerto Rico","state_population_size"] = 3725789

In [10]:
state_populations.state_population_size = state_populations.state_population_size.astype("int")

In [11]:
# quarantine_df_orig = quarantine_df_orig[['State', 'Coverage.location', 'Effective Date','population_size']]
# quarantine_df_orig = quarantine_df_orig.rename(columns={'State': 'Province_State', 'Effective Date': 'lockdown_date'})

In [12]:
# len(quarantine_df_orig)

In [13]:
# len(quarantine_df)

In [14]:
# pd.concat([quarantine_df,quarantine_df_orig], axis=1, join='inner')

In [15]:
# quarantine_df.join(quarantine_df_orig,on=["Province_State","lockdown_date","Coverage.location"])

In [16]:
quarantine_df = quarantine_df.merge(state_populations)

In [17]:
# quarantine_df["state_population_size"]/quarantine_df["state_population_size"]

In [18]:
# pd.merge(quarantine_df,quarantine_df_orig,on=["Province_State","lockdown_date","Coverage.location"])

In [19]:
severityScore = {'g':0.1, 's':0.5, 'r':0.4, 'n':0.7, 'l':1, 't':0.3, 'c':0.4, 'e':0.2}

In [20]:
# quarantine_df.apply(lambda x: int(x["population_size"])/int(state_populations_lookup[x["Province_State"]]),axis=1)

In [21]:
quarantine_df

,Province_State,Coverage,Coverage.location,lockdown_date,population_size,lockdown_type,emoji_string,emoji,event_index,state_population_size
0,Alabama,Statewide,Alabama,2020-03-13,4903185,Declaration of Emergency,e,⚠️,0,4903185
1,Alabama,Statewide,Alabama,2020-03-27,4903185,Closure of Non-essential Businesses,n,🏬,0,4903185
2,Alabama,Statewide,Alabama,2020-04-04,4903185,Stay-at-home Order,l,🏠,0,4903185
3,Alabama,Statewide,Alabama,2020-03-18,4903185,"Gatherings (>50) Banned, Closure of Schools",s,🎓,0,4903185
4,Alabama,Statewide,Alabama,2020-03-18,4903185,"Gatherings (>50) Banned, Closure of Schools",g,👨‍👩‍👧‍👦,1,4903185
...,...,...,...,...,...,...,...,...,...,...
769,Hawaii,Regional,Kauai,2020-03-20,72293,"Regional Border Closure/Visitor Quarantine, Cu...",l,🏠,0,1415872
770,Hawaii,Regional,Kauai,2020-03-20,72293,"Regional Border Closure/Visitor Quarantine, Cu...",c,🛩,1,1415872
771,Hawaii,Regional,Maui,2020-03-19,167417,Regional Closure of Restaurants,r,🍔,1,1415872
772,Iowa,Statewide,Iowa,2020-03-09,3155070,Declaration of Emergency,e,⚠️,0,3155070


In [22]:
# quarantine_df[quarantine_df["population_size"].isna()]
quarantine_df = quarantine_df.dropna(subset=["population_size"])

In [23]:
quarantine_df.population_size = quarantine_df.population_size.str.replace(",","")

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [24]:
quarantine_df.population_size = quarantine_df.population_size.astype("int")

In [25]:
quarantine_df["pctStateAffected"]= quarantine_df["population_size"]/quarantine_df["state_population_size"]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
assert all(quarantine_df[quarantine_df["pctStateAffected"]==1]["Coverage"]=="Statewide")

In [27]:
quarantine_df["severityScore"] = quarantine_df.emoji_string.apply(lambda x: severityScore[x])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
quarantine_df["interventionFootprint"] = quarantine_df["pctStateAffected"]*quarantine_df["severityScore"]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
quarantine_df.lockdown_date = quarantine_df.lockdown_date.str.replace("/","-")

In [46]:
# from chartlib import CovidChart, DaysSinceNumReached, days_between
# days_since = 20
# jhu_df = pd.read_csv('./data/jhu-data.csv')
# # grab us-specific
# jhu_df = jhu_df[(jhu_df.Country_Region == 'United States') & jhu_df.Province_State.notnull()]
# groupcol = 'Province_State'
# chart = CovidChart(
#     jhu_df,
#     groupcol=groupcol,
#     start_criterion=DaysSinceNumReached(days_since, 'Confirmed'),
#     ycol='interventionFootprint',
#     level='usa',
#     xcol='Date',
#     top_k_groups=30,
#     quarantine_df=quarantine_df
# )
# from datetime import datetime
# def first_alphabetic_group(df, groupcol):
#     return sorted(df[groupcol].unique())[0]
# EXTRA_DAYS_TO_INCLUDE = days_between('2020-04-28', datetime.now())
# chart.set_unfocused_opacity(0.05)
# chart = chart.set_ytitle('Number of Confirmed Cases (log scale)')
# chart = chart.set_xtitle(['Days since {} Confirmed'.format(days_since), '(Events prior to Day 0 not shown)'])
# chart.set_width(600).set_height(400)
# chart.set_xdomain((0, 47 + EXTRA_DAYS_TO_INCLUDE)).set_ydomain((days_since, 200000))
# chart.click_selection_init = first_alphabetic_group(chart._preprocess_df(), groupcol)

In [60]:
quarantine_df.lockdown_date = pd.to_datetime(quarantine_df.lockdown_date)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [391]:
state = "California"
qdf = quarantine_df[quarantine_df["Province_State"]==state]

In [394]:
qdf = qdf.groupby("lockdown_date").sum().reset_index()

In [395]:
qdf

,lockdown_date,population_size,event_index,state_population_size,pctStateAffected,severityScore,interventionFootprint
0,2020-03-04,39512223,0,39512223,1.000000,0.2,0.200000
1,2020-03-12,39512223,0,39512223,1.000000,0.1,0.100000
2,2020-03-13,10506571,55,434634453,0.265907,5.5,0.132953
3,2020-03-17,6659655,15,237073338,0.168547,6.0,0.168547
4,2020-03-19,39512223,0,39512223,1.000000,1.0,1.000000


In [397]:
qdf.interventionFootprint.cumsum()

0    0.200000
1    0.300000
2    0.432953
3    0.601500
4    1.601500
Name: interventionFootprint, dtype: float64

In [401]:
qdf = qdf.agg({'lockdown_date':lambda x: x,'interventionFootprint':'cumsum'})

In [197]:
# qdf.cumsum(axis="interventionFootprint")

In [257]:
df_all = []
for state in quarantine_df.Province_State.unique():
    qdf = quarantine_df[quarantine_df["Province_State"]==state]
    # qdf = qdf.groupby('lockdown_date').interventionFootprint.sum()
    qdf = qdf.groupby('lockdown_date').agg({
        'pctStateAffected': lambda col:sum(col),
        'emoji_string':lambda col: list(set(col))[0]
    }).reset_index()
    # make sure severity score is not summed
    qdf["interventionFootprint"] = qdf.apply(lambda x: x["pctStateAffected"]* severityScore[x["emoji_string"]],axis=1)
    
#     qdf = qdf.groupby('lockdown_date').interventionFootprint.sum()
#     qdf = qdf.groupby(qdf.index.loc).cumsum().reset_index()
#     qdf = qdf.groupby(qdf.index).cumsum().reset_index()
    qdf["State"]=state
    df_all.append(qdf)

In [258]:
qdf

,lockdown_date,pctStateAffected,emoji_string,interventionFootprint,State
0,2020-03-09,1.0,e,0.2,Iowa
1,2020-03-15,1.0,s,0.5,Iowa


In [229]:
df_all = pd.concat(df_all)

In [230]:
chart = alt.Chart(df_all).mark_line().encode(
        x = "lockdown_date",
        y = "interventionFootprint",
        facet=alt.Facet('State:O', columns=4),
    )
chart 

ValueError: lockdown_date encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.Chart(...)

In [212]:
import altair as alt
alt.Chart(qdf).mark_line().encode(
    x = "lockdown_date",
    y = "interventionFootprint"
).properties(title=state)

alt.Chart(...)

In [92]:
def plot_interventionFootprint(state):
    qdf = quarantine_df[quarantine_df["Province_State"]==state]
    qdf = qdf.groupby('lockdown_date').interventionFootprint.sum()
    qdf = qdf.groupby(qdf.index.month).cumsum().reset_index()
    chart = alt.Chart(qdf).mark_line().encode(
        x = "lockdown_date",
        y = "interventionFootprint"
    ).properties(title=state)
    return chart

In [110]:
for state in quarantine_df.Province_State.unique():
    display(plot_interventionFootprint(state))

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [256]:
quarantine_df

,Province_State,Coverage,Coverage.location,lockdown_date,population_size,lockdown_type,emoji_string,emoji,event_index,state_population_size,pctStateAffected,severityScore,interventionFootprint
0,Alabama,Statewide,Alabama,2020-03-13,4903185,Declaration of Emergency,e,⚠️,0,4903185,1.000000,0.2,0.200000
1,Alabama,Statewide,Alabama,2020-03-27,4903185,Closure of Non-essential Businesses,n,🏬,0,4903185,1.000000,0.7,0.700000
2,Alabama,Statewide,Alabama,2020-04-04,4903185,Stay-at-home Order,l,🏠,0,4903185,1.000000,1.0,1.000000
3,Alabama,Statewide,Alabama,2020-03-18,4903185,"Gatherings (>50) Banned, Closure of Schools",s,🎓,0,4903185,1.000000,0.5,0.500000
4,Alabama,Statewide,Alabama,2020-03-18,4903185,"Gatherings (>50) Banned, Closure of Schools",g,👨‍👩‍👧‍👦,1,4903185,1.000000,0.1,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,Hawaii,Regional,Kauai,2020-03-20,72293,"Regional Border Closure/Visitor Quarantine, Cu...",l,🏠,0,1415872,0.051059,1.0,0.051059
770,Hawaii,Regional,Kauai,2020-03-20,72293,"Regional Border Closure/Visitor Quarantine, Cu...",c,🛩,1,1415872,0.051059,0.4,0.020424
771,Hawaii,Regional,Maui,2020-03-19,167417,Regional Closure of Restaurants,r,🍔,1,1415872,0.118243,0.4,0.047297
772,Iowa,Statewide,Iowa,2020-03-09,3155070,Declaration of Emergency,e,⚠️,0,3155070,1.000000,0.2,0.200000


In [115]:
qdf

,lockdown_date,interventionFootprint,State
0,2020-03-09,0.2,Iowa
1,2020-03-15,0.7,Iowa


In [128]:
df_all = []
for state in quarantine_df.Province_State.unique():
    qdf = quarantine_df[quarantine_df["Province_State"]==state]
    qdf = qdf.groupby('lockdown_date').interventionFootprint.sum()
    qdf = qdf.groupby(qdf.index.month).cumsum().reset_index()
    qdf["State"]=state
    df_all.append(qdf)

In [129]:
df_all = pd.concat(df_all)

In [130]:
df_all

,lockdown_date,interventionFootprint,State
0,2020-03-13,0.200000,Alabama
1,2020-03-18,0.800000,Alabama
2,2020-03-24,0.842805,Alabama
3,2020-03-27,1.603853,Alabama
4,2020-04-04,1.000000,Alabama
...,...,...,...
4,2020-03-21,1.394105,Hawaii
5,2020-03-22,2.082418,Hawaii
6,2020-03-25,3.082418,Hawaii
0,2020-03-09,0.200000,Iowa


In [131]:
chart = alt.Chart(df_all).mark_line().encode(
        x = "lockdown_date",
        y = "interventionFootprint",
        facet=alt.Facet('State:O', columns=4),
    )
chart 

alt.Chart(...)